In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import matplotlib.pyplot as plt
import pandas as pd
from src.model import BasicModel

plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

class LEARN_INFO:
    def __init__(self, name, columns, is_contain_cluster_label=False, matching_type="general", jump=3):
        self.name = name
        self.columns = columns
        self.is_label = is_contain_cluster_label
        self.matching = matching_type
        self.jump = jump
    

# INPUT_REPORT_SIMULATION = [
#     LEARN_INFO(name="univariate", columns=['energy (kw 15min)'], is_contain_cluster_label=False),
#     LEARN_INFO(name="with cluster label", columns=['energy (kw 15min)'], is_contain_cluster_label=True),
# ]

DEFAULT_COLUMNS = [
    'energy (kw 15min)',
    ['week sin',
    'week cos'],
    ['year sin',
    'year cos'],
    'season idx',
    'weather idx',
    'avg ta',
    'avg rhm'
]

In [5]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos'
    ],
    "multivariate (year)": [
        'energy (kw 15min)',
        'year sin',
        'year cos',
    ],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
    "multivariate (avg ta)": [
        'energy (kw 15min)',
        'avg ta'
    ],
    "multivariate (avg rhm)": [
        'energy (kw 15min)',
        'avg rhm'
    ],
    "multivariate (season)": [
        'energy (kw 15min)',
        'season idx'
    ]
}

# Input Report Simulation
IRS = []

for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

for key in _.keys():
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )

# Training Model Array 
TMA = []

is_switch = True
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in IRS:
    print("--- now info ---")

    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch,
                   matching_type=_.matching,
                   is_multi_step=True,
                   model_type="Multi Step / Autoregressive")
    print(test_evaluate_df)
    print(val_evaluate_df)
    print()
    bm.set_window(OUT_STEPS=3)
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_multi_predict(is_reshape=True)
    bm.set_multi_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
    test_evaluate_df = test_evaluate_df.append(bm.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(bm.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (season) / with cluster) set predict (validation) info start ###### 

0 / 171
50 / 171
100 / 171
150 / 171
171 / 171 complete.

###### [Notice] set predict info success ###### 



In [3]:
test_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate,0.886071,61.657446,0.273971,0.011849,1.673844,0.879424,0.155560,1456.910039,558.616521
multivariate (week),0.894911,64.297860,0.265250,0.016220,1.588908,0.888980,0.144848,1456.910039,520.148067
multivariate (year),0.854531,52.156170,0.323473,0.014664,1.914025,0.836029,0.194108,1456.910039,697.041567
multivariate (weather),0.886415,61.812255,0.272452,0.017708,1.665786,0.880484,0.154932,1456.910039,556.361091
multivariate (avg ta),0.785247,20.813019,0.425057,0.014576,2.419159,0.795513,0.321271,1456.910039,1153.683072
multivariate (avg rhm),0.880602,60.042176,0.279183,0.015186,1.702672,0.873632,0.162114,1456.910039,582.149552
multivariate (season),0.868322,55.464562,0.287564,0.018851,1.779068,0.861739,0.180685,1456.910039,648.841262
univariate / with cluster,0.892480,56.090236,0.309729,0.010770,1.770386,0.893373,0.178147,1456.910039,639.725760
multivariate (week) / with cluster,0.889649,54.783627,0.310136,0.007879,1.759908,0.896286,0.183448,1456.910039,658.761882
multivariate (year) / with cluster,0.838416,46.482165,0.358626,0.009777,2.046550,0.812122,0.217128,1456.910039,779.706713


In [4]:
val_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate,0.905486,65.451574,0.261736,0.026887,1.674502,0.882190,0.150504,832.494756,287.613835
multivariate (week),0.913307,68.087141,0.258274,0.012200,1.586308,0.891672,0.139023,832.494756,265.672878
multivariate (year),0.880445,56.759584,0.319714,0.022241,1.911629,0.846556,0.188370,832.494756,359.974197
multivariate (weather),0.906116,65.686746,0.258985,0.026344,1.664733,0.883753,0.149480,832.494756,285.656038
multivariate (avg ta),0.890676,59.328634,0.280585,0.025397,1.796614,0.859294,0.177178,832.494756,338.586988
multivariate (avg rhm),0.907591,66.161952,0.258640,0.027260,1.656448,0.883719,0.147410,832.494756,281.699978
multivariate (season),0.899574,62.418820,0.270954,0.017821,1.730157,0.872179,0.163716,832.494756,312.861352
univariate / with cluster,0.929082,72.341471,0.259288,0.028540,1.514379,0.908670,0.120490,832.494756,230.255802
multivariate (week) / with cluster,0.921387,70.072097,0.279112,0.023930,1.571841,0.902991,0.130376,832.494756,249.148226
multivariate (year) / with cluster,0.852842,47.368893,0.404039,0.012428,2.128488,0.833027,0.229278,832.494756,438.151209


In [6]:
test_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate,0.907059,65.995445,0.262295,0.036501,1.664605,0.884490,0.148135,832.494756,283.086133
multivariate (week),0.912658,67.965186,0.257179,0.022566,1.588053,0.891384,0.139554,832.494756,266.688146
multivariate (year),0.884040,58.071135,0.312297,0.031069,1.890828,0.852249,0.182656,832.494756,349.055601
multivariate (weather),0.906270,65.740375,0.260651,0.033543,1.664996,0.883704,0.149246,832.494756,285.209582
multivariate (avg ta),0.889597,59.770198,0.286572,0.030097,1.795881,0.857876,0.175254,832.494756,334.910990
multivariate (avg rhm),0.906800,65.923613,0.258173,0.012913,1.660403,0.883234,0.148448,832.494756,283.684133
multivariate (season),0.902070,63.303031,0.263157,0.041456,1.704124,0.876128,0.159864,832.494756,305.500340
univariate / with cluster,0.899255,63.243535,0.316203,0.013787,1.764470,0.879941,0.160123,832.494756,305.995642
multivariate (week) / with cluster,0.895567,61.451367,0.329842,0.011522,1.795215,0.871392,0.167931,832.494756,320.915345
multivariate (year) / with cluster,0.874555,54.867124,0.368729,0.020414,1.977334,0.851731,0.196614,832.494756,375.728822


In [7]:
val_evaluate_df

,cos,ecv,mae,map,mean dis,mean sim,mse,tss,wss
univariate,0.884932,61.314184,0.275242,0.010915,1.680199,0.878425,0.156953,1456.910039,563.617534
multivariate (week),0.894836,64.455711,0.263188,0.013335,1.582914,0.888750,0.144207,1456.910039,517.848318
multivariate (year),0.850363,50.895951,0.326932,0.008081,1.938263,0.834549,0.199221,1456.910039,715.401825
multivariate (weather),0.885509,61.544764,0.274175,0.011723,1.673457,0.879251,0.156017,1456.910039,560.258191
multivariate (avg ta),0.794256,30.289703,0.368263,0.014948,2.224167,0.793697,0.282823,1456.910039,1015.616320
multivariate (avg rhm),0.879463,59.749537,0.280258,0.012702,1.709265,0.873140,0.163301,1456.910039,586.413034
multivariate (season),0.863716,54.162529,0.291587,0.014893,1.800264,0.858206,0.185968,1456.910039,667.810717
univariate / with cluster,0.875867,55.024249,0.317674,0.008204,1.819264,0.871031,0.182472,1456.910039,655.256236
multivariate (week) / with cluster,0.876003,53.152439,0.328165,0.007828,1.844387,0.871323,0.190066,1456.910039,682.526823
multivariate (year) / with cluster,0.865219,51.163062,0.347828,0.005541,1.959131,0.842598,0.198137,1456.910039,711.510254


In [11]:
TMA[0].norm_datas['val']

,energy (kw 15min)
Date Time,
2018-04-02 00:00:00,-0.774395
2018-04-02 01:00:00,-0.829437
2018-04-02 02:00:00,-0.847784
2018-04-02 03:00:00,-0.847784
2018-04-02 04:00:00,-0.829437
...,...
2018-09-19 19:00:00,-0.187287
2018-09-19 20:00:00,-0.425800
2018-09-19 21:00:00,-0.352411
